In [1]:
import os
import javalang
import pickle
from gensim.models.word2vec import Word2Vec
import numpy as np
import re
import string
import random
import sys

In [9]:
def solve_camel_and_underline(token):
	if token.isdigit():
		return [token]
	else:
		p = re.compile(r'([a-z]|\d)([A-Z])')
		sub = re.sub(p, r'\1_\2', token).lower() #lowercase
		sub_tokens = sub.split("_")
		tokens = re.sub(" +", " ", " ".join(sub_tokens)).strip()
		final_token = []
		for factor in tokens.split(" "):
			final_token.append(factor.rstrip(string.digits))
		return final_token


In [3]:
def cut_data(token_seq, token_length_for_reserve):
	if len(token_seq) <= token_length_for_reserve:
		return token_seq
	else:
		start_index = token_seq.index("rank2fixstart")
		end_index = token_seq.index("rank2fixend")
		assert end_index > start_index
		length_of_annotated_statement = end_index - start_index + 1
		if length_of_annotated_statement <= token_length_for_reserve:
			padding_length = token_length_for_reserve - length_of_annotated_statement
			# give 2/3 padding space to content before annotated statement
			pre_padding_length = padding_length // 3 * 2
			# give 1/3 padding space to content after annotated statement
			post_padding_length = padding_length - pre_padding_length
			if start_index >= pre_padding_length and len(token_seq) - end_index - 1 >= post_padding_length:
				return token_seq[start_index - pre_padding_length: end_index + 1 + post_padding_length]
			elif start_index < pre_padding_length:
				return token_seq[:token_length_for_reserve]
			elif len(token_seq) - end_index - 1 < post_padding_length:
				return token_seq[len(token_seq) - token_length_for_reserve:]
		else:
			return token_seq[start_index: start_index + token_length_for_reserve]

In [29]:
method='public void method1(int i){\n rank2fixstart String AaBoy=new String("123"); rank2fixstart\nint b=1;\nSystem.out.println(a);\n}'

In [30]:
tokens=javalang.tokenizer.tokenize(method)

In [49]:
# for token in tokens:
# 	print(token)

Modifier "public" line 1, position 0
Keyword "void" line 1, position 7
Identifier "method1" line 1, position 12
Separator "(" line 1, position 19
BasicType "int" line 1, position 20
Identifier "i" line 1, position 24
Separator ")" line 1, position 25
Separator "{" line 1, position 26
Identifier "String" line 2, position 2
Identifier "AaBoy" line 2, position 9
Operator "=" line 2, position 14
Keyword "new" line 2, position 15
Identifier "String" line 2, position 19
Separator "(" line 2, position 25
String ""123"" line 2, position 26
Separator ")" line 2, position 31
Separator ";" line 2, position 32
BasicType "int" line 3, position 1
Identifier "b" line 3, position 5
Operator "=" line 3, position 6
DecimalInteger "1" line 3, position 7
Separator ";" line 3, position 8
Identifier "System" line 4, position 1
Separator "." line 4, position 7
Identifier "out" line 4, position 8
Separator "." line 4, position 11
Identifier "println" line 4, position 12
Separator "(" line 4, position 19
Ident

In [31]:
def generate_token_seq(tokens):
	last_token=None
	t_seq = []
	for token in tokens:
		if isinstance(token, javalang.tokenizer.String):
			tmp_token = ["stringliteral"]
		else:
			tmp_token = solve_camel_and_underline(token.value)
			if last_token is not None and isinstance(token, javalang.tokenizer.Identifier) and (
					isinstance(last_token, javalang.tokenizer.Identifier) or isinstance(last_token,
																						   javalang.tokenizer.BasicType)) and last_token.value!='rank2fixstart':
				cur_token=tmp_token
				tmp_token=[]
				type_token_seq = solve_camel_and_underline(last_token.value)
				for t_token in cur_token:
					tmp_token.append([type_token_seq, cur_token[:cur_token.index(t_token)], t_token])
		last_token=token
		t_seq += tmp_token
	return t_seq

In [32]:
t_s=generate_token_seq(tokens)

In [33]:
t_s

['public',
 'void',
 'method',
 '(',
 'int',
 [['int'], [], 'i'],
 ')',
 '{',
 'rank2fixstart',
 'string',
 [['string'], [], 'aa'],
 [['string'], ['aa'], 'boy'],
 '=',
 'new',
 'string',
 '(',
 'stringliteral',
 ')',
 ';',
 'rank2fixstart',
 'int',
 [['int'], [], 'b'],
 '=',
 '1',
 ';',
 'system',
 '.',
 'out',
 '.',
 'println',
 '(',
 'a',
 ')',
 ';',
 '}']